In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import LSTM as lstm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

In [2]:
path = "../nusax/datasets/sentiment/indonesian/"

train_data = pd.read_csv(path+"train.csv")
test_data = pd.read_csv(path+"test.csv")

# Text extraction
X_train_text = train_data['text'].astype(str).values
y_train = train_data['label'].values

X_test_text = test_data['text'].astype(str).values
y_test = test_data['label'].values

# Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Configure tokenizer
vocab_size = 2796
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Tokenization
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# Padding
max_len = 77
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [3]:
model = load_model('model_cell_3.keras')

embedding_weights = model.layers[0].get_weights()[0]
lstm_weights = model.layers[1].get_weights()
dense_weights_1 = model.layers[3].get_weights()
dense_weights_2 = model.layers[4].get_weights()
model.summary()

for i, layer in enumerate(model.layers):
    if hasattr(layer, 'activation'):
        print(f"Layer {i}: {layer.name}, Activation: {layer.activation.__name__}")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 77, 128)        │       357,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,306,315 (8.80 MB)

 Trainable params: 768,771 (2.93 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,537,544 (5.87 MB)

Layer 1: lstm_11, Activation: tanh
Layer 3: dense_10, Activation: relu
Layer 4: dense_11, Activation: softmax


In [4]:
import importlib
import LSTM
import Embedding
import Dense
import Model
importlib.reload(Dense)

relu = lambda x: np.maximum(0, x)
softmax = lambda x: np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)
model_scratch = Model.Model(
    layers=[
            Embedding.ScratchEmbedding(
            embedding_weight=embedding_weights
        ),

            LSTM.ScratchLSTM(
            hidden_size=256,
            lstm_weights=lstm_weights,
            manytomany=False
        ),

            Dense.DenseScratch(
            dense_weights=dense_weights_1,
            activation=relu
        ),

            Dense.DenseScratch(
            dense_weights=dense_weights_2,
            activation=softmax
        )
    ]
)



test = model_scratch.predict(X_train_pad[:10])
print(test) 
print(model.predict(X_train_pad[:10]))

DenseScratch initialized with weights shape: 256
DenseScratch initialized with weights shape: 64
[[0.33284244 0.54029414 0.12686342]
 [0.38769475 0.21177228 0.40053297]
 [0.33281599 0.54036653 0.12681748]
 [0.38784462 0.2119244  0.40023098]
 [0.38768421 0.2117789  0.40053689]
 [0.33291186 0.54013816 0.12694997]
 [0.38763053 0.21183591 0.40053355]
 [0.38769291 0.21177273 0.40053437]
 [0.38767321 0.21178918 0.40053762]
 [0.38769868 0.21177199 0.40052934]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
[[0.33284247 0.540294   0.12686352]
 [0.3876948  0.2117723  0.4005329 ]
 [0.33281597 0.5403665  0.12681757]
 [0.38784462 0.21192446 0.400231  ]
 [0.38768417 0.21177886 0.40053692]
 [0.33291185 0.54013807 0.12695007]
 [0.38763052 0.21183594 0.40053356]
 [0.38769287 0.2117728  0.4005343 ]
 [0.38767323 0.21178916 0.4005376 ]
 [0.3876986  0.21177197 0.40052944]]
